In [1]:
%cd src

/home/romet/projects/ut/automl/src


In [2]:
import pandas as pd
from dataloader import load_data

from mcnemar_test import create_contingency_table, calculate_mcnemar_test

%load_ext autoreload
%autoreload 2

In [3]:
train = load_data()
true_pred = train.target.values

Reading training data...
Original train shape:  (200000, 202)
Frequency encoding...
shape returned by frequency_encoding function (200000, 200)
Scaling data...
target data type is int64
shape returned by load_data function (200000, 402)


## LightGBM Optimized McNemar test

In [4]:
baseline_lgbm_df = pd.read_csv("../kfolds/LightGBM__folds5__0.8949656737461088.csv")
optimized_lgbm_df = pd.read_csv("../kfolds/LightGBMOptimized__folds5__0.9017873558406069.csv")

baseline_lgbm_pred = baseline_lgbm_df['target'] >= 0.5
optimized_lgbm_pred = optimized_lgbm_df['target'] >= 0.5

In [5]:
cont_lgbm_df = create_contingency_table(true_pred, baseline_lgbm_pred, optimized_lgbm_pred)
cont_lgbm_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,168204,3845
nr_incorrect_optimized,11190,16761


In [6]:
calculate_mcnemar_test(cont_lgbm_df)

statistic=3845.00000, p-value=0.00000e+00
Different proportions of errors (reject H0)


## LightGBM Autofeat Mcnemar Test


In [7]:
autofeat_lgbm_df = pd.read_csv("../kfolds/LightGBMAutoFeat__folds5__0.9014404790895103.csv")
autofeat_lgbm_pred = autofeat_lgbm_df['target'] >= 0.5

In [8]:
cont_lgbm_df = create_contingency_table(true_pred, baseline_lgbm_pred, autofeat_lgbm_pred)
cont_lgbm_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,166143,5906
nr_incorrect_optimized,7038,20913


In [9]:
calculate_mcnemar_test(cont_lgbm_df)

statistic=5906.00000, p-value=2.59278e-23
Different proportions of errors (reject H0)


## XGBoost Optimized McNemar test

In [10]:
baseline_xgb_df = pd.read_csv("../kfolds/XGBoost__folds5__0.8644135937992727.csv")
optimized_xgb_df = pd.read_csv("../kfolds/XGBoostOptimized__folds5__0.9029208825593404.csv")

In [11]:
baseline_xgb_pred = baseline_xgb_df['target'] >= 0.5
optimized_xgb_pred = optimized_xgb_df['target'] >= 0.5

In [12]:
cont_xgb_df = create_contingency_table(true_pred, baseline_xgb_pred, optimized_xgb_pred)
cont_xgb_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,162078,13895
nr_incorrect_optimized,8494,15533


In [13]:
calculate_mcnemar_test(cont_xgb_df)

statistic=8494.00000, p-value=5.22717e-288
Different proportions of errors (reject H0)


## XGBoost Autofeat McNemar test

In [14]:
autofeat_xgb_df = pd.read_csv("../kfolds/XGBoostAutoFeat__folds5__0.9021793206340705.csv")
autofeat_xgb_pred = autofeat_xgb_df['target'] >= 0.5

In [15]:
cont_xgb_df = create_contingency_table(true_pred, baseline_xgb_pred, autofeat_xgb_pred)
cont_xgb_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,161917,14056
nr_incorrect_optimized,8390,15637


In [16]:
calculate_mcnemar_test(cont_xgb_df)

statistic=8390.00000, p-value=2.98019e-316
Different proportions of errors (reject H0)
